# Figure 3 & 4 - The depolarisation of existing echo chambers by an RDN


## Figure 3 
The opinions of 1000 agents over time (a) without the RDN until 𝑡 = 10, with the RDN until 𝑡 = 20 (𝐷 = 3, black bar), and finally without the RDN until 𝑡 = 30. Kernel density estimates of agent opinions (5 trials each) at moments in time (b) at 𝑡 = 10 ("Pre RDN", pink), 𝑡 = 20 ("Post RDN", D ∈ 1,3,5, blue), and 𝑡 = 30 ("RDN off", tan). The peak
distance ($Λ_𝑥$) decreases when a RDN is applied. Other parameters were 𝑚 = 10, 𝐾 = 3, 𝛼 = 3, 𝛽 = 3, 𝑛 = 30, and 𝑟 = 0.5

## Figure 4
Figure 4. The effect of RDN on an agent’s opinion and interactions in the network. From a
simulation in Fig. 3 (𝐷 = 3), each agent 𝑖’s opinion 𝑥 is shown at 𝑡 = 10 ("Pre RDN", pink) and at 𝑡 = 20
("Post RDN", light blue) (a), as well as their change in opinion, Δ𝑥, (b). The incoming interactions from
other agents to an agent 𝑖 ("in degree", → 𝑖) is shown in (c) as a probability histogram 𝑃(→ 𝑖). The
density of in degree change Δ → 𝑖 as a function of Δ𝑥 (d). The interactions between agents before the
RDN (e, top panel) show a large number of interactions and small groups. That is in contrast to after the
RDN (e, bottom panel), which shows fewer interactions but larger groups. Note that activity probabilities
$𝑎_𝑖$ remained the same throughout the simulation. Other parameters were 𝑚 = 10, 𝐾 = 3, 𝛼 = 3, 𝛽 = 3,
𝑛 = 30, and 𝑟 = 0.5.

## Imports and settings


In [ ]:
#@title Colab Setup { display-mode: "form" }
import sys

# for persisting data across sessions
connect_gdrive = False #@param {type:"boolean"}
cache_dir = ".cache" #param {allow-input: true}

try:
    from google.colab import drive

    # Clone github repository
    GIT_REPO = "https://github.com/ChrisCurrin/opinion_dynamics.git"
    !rm -rf ./temp
    !git clone --quiet "{GIT_REPO}" ./temp

    # add to path
    sys.path.append('./temp')
    # need latest tqdm version for tenumerate and tables to prevent HDF read error
    # !pip install -q --upgrade tqdm tables
    # !pip install -q --upgrade vaex IPython

    if connect_gdrive:
        # connect GDrive for retrieving/saving results
        drive.mount('/content/drive')

        # create symlink between a Drive folder and the cache for persistence between sessions
        import os
        try:
            os.makedirs("/content/drive/My Drive/Colab Notebooks/opdynamics/.cache")
            os.makedirs("/content/drive/My Drive/Colab Notebooks/opdynamics/output")
        except IOError:
            pass
        !ln -s "/content/drive/My Drive/Colab Notebooks/opdynamics/.cache" ".cache"
        !ln -s "/content/drive/My Drive/Colab Notebooks/opdynamics/output" "output"

except ModuleNotFoundError:
    print(f"local notebook on {sys.platform}")
    pass

In [ ]:
#@title Imports and settings { display-mode: "form" }
%reload_ext autoreload
%autoreload 2

dark = True # @param {type:"bool"}

import logging
import itertools
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from tqdm.notebook import tqdm
from matplotlib.colors import LogNorm
from matplotlib.cbook import flatten
from matplotlib.collections import PolyCollection

try:
    import opdynamics.simulation as Simulation
except AttributeError:
    raise RuntimeError("restart runtime")
from opdynamics.socialnetworks import (SocialNetwork, ConnChamber, ContrastChamber, OpenChamber, SampleChamber)
from opdynamics.visualise import (VisSocialNetwork,
                                  show_K_alpha_phase,
                                  show_activity_vs_opinion,
                                  show_jointplot,
                                  show_noise_panel,
                                  show_opinion_grid,
                                  )
from opdynamics.metrics.opinions import calc_distribution_differences, distribution_modality
from opdynamics.visualise import plot_surface_product, plot_surfaces
from opdynamics.utils.distributions import negpowerlaw
from opdynamics.utils.plot_utils import df_multi_mask, move_cbar_label_to_title
from opdynamics.utils.cache import get_cache_dir, set_cache_dir, process_cache_arg
from opdynamics.utils.logging import LoggingContext
from opdynamics.utils.constants import *
from opdynamics.visualise.compat import sns_kdeplot

try:
    os.makedirs("output")
except FileExistsError:
    pass

old_cache_dir, new_dir = set_cache_dir(cache_dir)

np.random.seed(1337)
sns.set_theme(context="notebook", style="ticks",
        rc={
            "pdf.fonttype": 42, # embed font in output
            "svg.fonttype": "none",  # embed font in output
            # "figure.facecolor":"white",
            
            "axes.spines.left": True,
            "axes.spines.bottom": True,
            "axes.spines.right": False,
            "axes.spines.top": False,
            "figure.dpi": 200,
            }
        )

if dark:
    plt.style.use("dark_background")

# some settings get reset by dark_background
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["axes.facecolor"] = "None"

logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger("matplotlib").setLevel(logging.INFO)
logging.getLogger("asyncio").setLevel(logging.WARNING)

## Delayed noise

uses the `run_periodic_noise` function to start the noise after some time.

Note the use of `periodic` does indeed mean noise can be applied as a frequency. This can be specified using `num` and `interval`. Although, this isn't investigated in the paper, it is a useful feature.


In [ ]:
#@title Playground
%matplotlib inline

noise_start = 10.
noise_length = noise_start
recovery = noise_length
num = 1 #@param {type:"integer"}
interval = 0 #@param {type:"slider", min:0, max:10, step:0.5}
time_points = [noise_start] + list(np.round(np.arange(
    noise_start+noise_length, 
    noise_start+noise_length*2+recovery, 
    noise_length), 2)
    )

# Nudge arguments

D = 3 #@param {type:"slider", min:0, max:5, step:0.5}
sample_size = 30 #@param {type:"slider", min:0, max:50, step:1}
sample_method = "full" #@param ["full", "subsample", "simple"]
seed = "1572038" #@param ["1337", "10725983", "7988657", "1572038", "15379820"] {allow-input: true}

# polar opinions
N = 1000 #@param {type:"integer"}
m = 10 #@param {type:"integer"}
α = 3 #@param {type:"slider", min:0, max:5, step:0.5}
K = 3 #@param {type:"slider", min:0, max:5, step:0.5}
ε = 0.01 #@param {type:"slider", min:0.005, max:0.05, step:0.005}
Υ = 2.1 #@param {type:"slider", min:0, max:3, step:0.1}
β = 3 #@param {type:"slider", min:0, max:5, step:0.5}
r = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}
dt = 0.01

integration_method = "RK45" #@param ["Euler", "RK45", "Euler-Maruyama"] {allow-input: true}

cls = "SampleChamber" #@param ["SampleChamber", "OpenChamber"]

# could also use eval(cls)
cls_pick = {"SampleChamber": SampleChamber, "OpenChamber": OpenChamber}[cls]

kwargs = dict(N=N,
              m=m,
              activity_distribution=negpowerlaw,
              epsilon=ε,
              gamma=Υ,
              dt=dt,
              K=K,
              beta=β,
              alpha=α,
              r=r,
              cls=cls_pick,
              method=integration_method,
              D=D,
              sample_size=sample_size,
              sample_method=sample_method,
              noise_start=noise_start,
              noise_length=noise_length,
              recovery=recovery,
              interval=interval,
              num=num,
              seed=int(seed)
             )

lims = (-5,5)

with LoggingContext(logging.INFO):
  # note that plot_kws can be passed to customise the plots
  nsn = Simulation.run_periodic_noise(**kwargs, 
                                      cache="all", 
                                      plot_opinion=True, 
                                      plot_kws={
                                        "time_points": time_points, 
                                        "dist_kwargs": {"kind":"hist", "bins": np.linspace(-5.1, 5.1, 100)}, 
                                        "lims": lims}
                                      )
# annotate the plot with the peak distance
fig = plt.gcf()
time_ax, *dist_axs = fig.axes
time_ax.set_ylim(*lims)
for t, ax in zip(time_points, dist_axs):
    pop_dist = nsn.get_distribution_modality((t-0.0, t))
    ax.set_title("")
    ax.set_xlim(*lims)
    y_max = ax.get_ylim()[1]
    color='grey'
    ax.annotate(f"{pop_dist:.2f}",
                xy=(0, y_max),
                xytext=(0, y_max*1.05),
                arrowprops=dict(arrowstyle=f'-[,widthB={pop_dist}',
                                color='k', alpha=0.7, lw=0.5),
                  alpha=0.8,
                fontsize='x-small',
                va='bottom',
                ha='center')

    if t <= noise_start:
        ax.annotate(PEAK_DISTANCE, xy=(0,1), xycoords='axes fraction',
                    va='bottom', ha='right', fontsize='small')
fig.savefig(f"output/D={D}_seed={seed}_T={noise_start+noise_length+recovery}_delayed_noise_sample.svg")

## See how interactions changed

In [ ]:
# 

kwargs = {}


fig, axes = plt.subplots(ncols=2, sharex=False, sharey=True, 
                            gridspec_kw={"width_ratios": [1, 0.5]},
                            figsize=(6, 3))

t_point = float(noise_start)

t_idx = get_time_point_idx(nsn.result.t, t_point)
t_idx_end = get_time_point_idx(nsn.result.t, t_point+noise_length)

pre_noise_opinions_unsorted = nsn.result.y[:, t_idx]
post_noise_opinions_unsorted = nsn.result.y[:, t_idx_end]

x_idx = np.argsort(pre_noise_opinions_unsorted)

pre_noise_opinions = pre_noise_opinions_unsorted[x_idx]
post_noise_opinions = nsn.result.y[x_idx, t_idx_end]
change_in_opinions = post_noise_opinions - pre_noise_opinions
change_in_opinions_overall = np.sort(post_noise_opinions) - pre_noise_opinions

cmap_pre = sns.light_palette(PRE_RDN_COLOR, n_colors=1, as_cmap=True)
cmap_post = sns.light_palette(POST_RDN_COLOR, n_colors=1, as_cmap=True)
# green and purple diverging colormap (white in middle)
cmap_change = sns.diverging_palette(145, 300, s=-60, as_cmap=True)
cmap_change = sns.light_palette(DELTA_RDN_COLOR, as_cmap=True)

vmax = np.max(np.abs(change_in_opinions))
vmin = -vmax
v = vis._get_equal_opinion_limits()

sm_change = ScalarMappable(norm=Normalize(vmin, vmax), cmap=cmap_change)

vis.show_agent_opinions(sort=x_idx, t=t_point, title=False, ax=axes[0],
                        colorbar=False, show_middle=False,
                        alpha=0.3,
                        cmap=cmap_pre,
                        )
vis.show_agent_opinions(sort=x_idx, t=t_point+noise_length, title=False, ax=axes[0], 
                        alpha=0.7, colorbar=False, show_middle=False,
                        cmap=cmap_post
                        )

axes[1].barh(
    np.arange(nsn.N),
    change_in_opinions,
    color=sm_change.to_rgba(np.abs(change_in_opinions)),
    edgecolor="None",
    linewidth=0,  # remove bar borders
    height=1,  # per agent
    alpha=0.8,
    **kwargs
)
#     cbar = colorbar_inset(sm_change, "outer bottom", size="5%", pad=0.01, ax=axes[1])
axes[0].set_xlim(-6,6)
axes[1].set_xlim(-2,2)

#     axes[1].set_ylim(0, nsn.N)
# axes[0].set_xscale("symlog")
# axes[1].set_xscale("symlog")

for ax in axes:
    sns.despine(ax=ax, offset=5)
#         ax.tick_params(axis="x", bottom=False, labelbottom=False)
axes[1].set_xlabel(math_fix(f"$\Delta ${OPINION_SYMBOL}"))
axes[0].legend([Line2D([],[], color=PRE_RDN_COLOR, alpha=0.3), Line2D([],[], color=POST_RDN_COLOR, alpha=0.7)],
                ["Pre RDN", "Post RDN"], 
                frameon=False)
fig.savefig(f"output/D={D}_seed={seed}_dx.svg")

In [ ]:
#@title Opinions before and after RDN
from matplotlib.lines import Line2D
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize, SymLogNorm

from opdynamics.utils.plot_utils import colorbar_inset, colorline, get_time_point_idx

vis = VisSocialNetwork(nsn)

fig, axes = plt.subplots(ncols=2, sharex=False, sharey=True, 
                             gridspec_kw={"width_ratios": [1, 0.5]})

sort_post = True # @param {type:"bool"}

t_point = float(noise_start)

t_idx = get_time_point_idx(nsn.result.t, t_point)
t_idx_end = get_time_point_idx(nsn.result.t, t_point+noise_length)

pre_noise_opinions_unsorted = nsn.result.y[:, t_idx]
post_noise_opinions_unsorted = nsn.result.y[:, t_idx_end]

x_idx = np.argsort(pre_noise_opinions_unsorted)

pre_noise_opinions = pre_noise_opinions_unsorted[x_idx]
post_noise_opinions = nsn.result.y[x_idx, t_idx_end]
change_in_opinions = post_noise_opinions - pre_noise_opinions
change_in_opinions_overall = np.sort(post_noise_opinions) - pre_noise_opinions

vis.show_agent_opinions(sort=True if sort_post else x_idx, t=t_point, title=False, ax=axes[0],
                        colorbar=False, show_middle=False,
                        alpha=0.3,
                        cmap=cmap_pre,
                        )
vis.show_agent_opinions(sort=True if sort_post else x_idx, t=t_point+noise_length, title=False, ax=axes[0], 
                        alpha=0.7, colorbar=False, show_middle=False,
                        cmap=cmap_post
                        )

# vis.show_agent_opinions(sort=True, t=t_point, title=False, ax=axes[0],
#                         colorbar=False, show_middle=False,
#                         alpha=0.3,
#                         cmap=cmap_pre,
#                         )
# vis.show_agent_opinions(sort=True, t=t_point+noise_length, title=False, ax=axes[0], 
#                         alpha=0.7, colorbar=False, show_middle=False,
#                         cmap=cmap_post
#                         )


change_in_opinions = post_noise_opinions_unsorted - pre_noise_opinions_unsorted
change_in_opinions_sorted = np.sort(post_noise_opinions_unsorted)-np.sort(pre_noise_opinions_unsorted)
cmap_change = sns.diverging_palette(145, 300, s=-60, as_cmap=True)
vmax = np.max(np.abs(change_in_opinions))
vmin = -vmax
v = vis._get_equal_opinion_limits()

change_in_opinions_to_use = change_in_opinions_sorted if sort_post else change_in_opinions

mask = ((change_in_opinions_to_use>0) & (np.sort(pre_noise_opinions_unsorted)>0)) | ((change_in_opinions_to_use<0) & (np.sort(pre_noise_opinions_unsorted)<0))

sm_change = ScalarMappable(norm=SymLogNorm(linthresh=0.1, vmin=vmin, vmax=vmax), cmap=cmap_change)

direction = sm_change.to_rgba(change_in_opinions_to_use)

axes[1].barh(
    np.array(nsn.agent_idxs)[mask],
    change_in_opinions_to_use[mask],
    edgecolor="None",
    linewidth=0,  # remove bar borders
    height=1,  # per agent
    alpha=0.5,
    color='grey'
)
axes[1].barh(
    np.array(nsn.agent_idxs)[~mask],
    change_in_opinions_to_use[~mask],
    edgecolor="None",
    linewidth=0,  # remove bar borders
    height=1,  # per agent
    alpha=0.5,
    color=DELTA_RDN_COLOR
)
axes[1].set_xlabel(math_fix(f"$\Delta ${OPINION_SYMBOL}"))
axes[0].legend([Line2D([],[], color=PRE_RDN_COLOR, alpha=0.3), Line2D([],[], color=POST_RDN_COLOR, alpha=0.7)],
                ["Pre RDN", "Post RDN"], 
                frameon=False, title="Network state")
axes[1].legend([Line2D([],[], color=DELTA_RDN_COLOR), Line2D([],[], color='grey')],
                ["More neutral", "More polarized"], 
                frameon=False)
axes[0].set_xlim(-8,8)
axes[1].set_xlim(-3,3)
plt.savefig(f"output/D={D}_seed={seed}_dx_{'sorted' if sort_post else ''}.svg")


In [ ]:
fig, ax_joint, ax_marg_x, ax_marg_y = vis.show_nearest_neighbour(t=noise_start, color=PRE_RDN_COLOR)
ax_joint.set_xlim(-5, 5)
ax_joint.set_ylim(-5, 5)
fig.savefig(f"output/D={D}_seed={seed}_nn_pre.svg")
fig, ax_joint, ax_marg_x, ax_marg_y = vis.show_nearest_neighbour(t=noise_start+noise_length, color=POST_RDN_COLOR)
ax_joint.set_xlim(-5, 5)
ax_joint.set_ylim(-5, 5)
fig.savefig(f"output/D={D}_seed={seed}_nn_post.svg")

In [ ]:
# get network properties

print("[0; noise start]")
G_start, df_start = nsn.get_network_agents((0.0, noise_start))
_, df_start_conn = nsn.get_network_connections((0.0, noise_start))
df_start["RDN"] = "Pre RDN"
df_start_conn["RDN"] = "Pre RDN"

print("[noise start; noise end]")
G_noise, df_noise = nsn.get_network_agents((noise_start, noise_start + noise_length))
_, df_noise_conn = nsn.get_network_connections(
    (noise_start, noise_start + noise_length)
)
df_noise["RDN"] = "Post RDN"
df_noise_conn["RDN"] = "Post RDN"

print("[0, noise end]")
G_full_noise, df_full_noise = nsn.get_network_agents((0.0, noise_start + noise_length))
_, df_full_noise_conn = nsn.get_network_connections((0.0, noise_start + noise_length))

df_degree = pd.concat([df_start, df_noise], ignore_index=True)
df_conn = pd.concat([df_start_conn, df_noise_conn], ignore_index=True)

df_degree


In [ ]:
rdn_name = "Network state"
density_label = math_fix(f"P({OPINION_SYMBOL},{IN_DEGREE_SYMBOL})")

with sns.plotting_context("paper"):
    plt.rcParams["legend.frameon"] = False

    fig, ax = plt.subplots(figsize=(3, 3), dpi=200)

    inset_ax = ax.inset_axes(bounds=[0.7, 0.3, 0.3, 0.3])

    sns.histplot(
        df_degree.rename(columns={"RDN": rdn_name}),
        x="in_degree",
        hue=rdn_name,
        palette=[PRE_RDN_COLOR, POST_RDN_COLOR],
        log_scale=True,
        stat="probability",
        ax=ax,
    )

    ax.set(
        xlabel=f"In degree for agent $i$ \n " + math_fix(f"[{IN_DEGREE_SYMBOL}]"),
        ylabel=math_fix(f"P({IN_DEGREE_SYMBOL})"),
    )

    sns.kdeplot(
        data=df_degree.rename(columns={"RDN": rdn_name}),
        x="opinion",
        y="in_degree",
        hue=rdn_name,
        palette=[PRE_RDN_COLOR, POST_RDN_COLOR],
        alpha=0.5,
        levels=8,
        log_scale=(False, True),
        fill=True,
        legend=False,
        ax=inset_ax,
    )

    inset_ax.set(
        xlim=(-5, 5),
        ylim=(40, 5000),
        ylabel=IN_DEGREE_SYMBOL,
        xlabel=OPINION_SYMBOL,
        title=density_label,
    )
    plt.savefig(f"output/D={D}_seed={seed}_in_degree.svg")


In [ ]:
from matplotlib.colors import LogNorm, Normalize, SymLogNorm
from opdynamics.visualise.dense import show_matrix
from opdynamics.utils.plot_utils import get_time_point_idx

window_size = noise_start / 2

t_idx_pre, opinions_pre_rdn_all = nsn.opinions_at_t(
    (noise_start - window_size, noise_start), flatten=False
)
t_idx_post, opinions_post_rdn_all = nsn.opinions_at_t(
    ((noise_start + noise_length - window_size), noise_start + noise_length),
    flatten=False,
)

acc_adj_mat_pre_rdn = nsn.adj_mat.accumulate(t_idx_pre)
acc_adj_mat_post_rdn = nsn.adj_mat.accumulate(t_idx_post)

opinions_pre_rdn = opinions_pre_rdn_all[:, -1]
opinions_post_rdn = opinions_post_rdn_all[:, -1]

agent_mat_pre = pd.DataFrame(
    acc_adj_mat_pre_rdn,
    columns=pd.Index(nsn.agent_idxs, name="i"),
    index=pd.Index(nsn.agent_idxs, name="j"),
)
agent_mat_post = pd.DataFrame(
    acc_adj_mat_post_rdn,
    columns=pd.Index(nsn.agent_idxs, name="i"),
    index=pd.Index(nsn.agent_idxs, name="j"),
)

with sns.plotting_context("paper"):

    fig, axes = plt.subplots(
        nrows=3, sharex=True, sharey=True, figsize=(3, 4.5), dpi=400
    )
    fig.subplots_adjust(left=0.3, right=0.7, hspace=0.1)
    cbar_kws = dict(
        vmin=1, 
        # vmax=max(agent_mat_pre.values.max(), agent_mat_post.values.max())
    )
    cbar_kws["vmax"] = 20
    fig, ax = show_matrix(
        agent_mat_pre,
        "Pre RDN interactions",
        map="mesh",
        sort=True,
        #                       cmap="viridis",
        cmap=sns.dark_palette(PRE_RDN_COLOR, as_cmap=True, input="hex"),
        title="",
        rasterized=True,
        ax=axes[0],
        **cbar_kws
    )
    axes[0].set_xlabel("")
    # access computed sorted dataframe (not present if sort=False)
    df_sorted_mat = show_matrix.sorted_mat

    # set to same index/columns as pre-rdn sorted matrix
    agent_mat_post = agent_mat_post.loc[df_sorted_mat.index, df_sorted_mat.columns]
    fig, ax = show_matrix(
        agent_mat_post,
        "Post RDN interactions",
        map="mesh",
        sort=False,
        #                       cmap="viridis",
        # cmap=sns.blend_palette(['#e5e5e5', POST_RDN_COLOR], as_cmap=True, input="hex"),
        cmap=sns.dark_palette(POST_RDN_COLOR, as_cmap=True),
        title="",
        rasterized=True,
        ax=axes[1],
        zorder=-10,
        **cbar_kws
    )

    delta_mat = agent_mat_post - agent_mat_pre
    delta_mat_abs = np.abs(delta_mat)
    fig, ax = show_matrix(
        delta_mat,
        "Delta interactions",
        map="mesh",
        sort=False,
        cmap="coolwarm",
        #   cmap=sns.light_palette(DELTA_RDN_COLOR, as_cmap=True),
        title="",
        rasterized=True,
        ax=axes[-1],
        #  norm=LogNorm(), vmin=1, vmax=delta_mat_abs.values.max(),
        norm=SymLogNorm(
            1, vmin=-delta_mat_abs.values.max(), vmax=delta_mat_abs.values.max()
        ),
    )

    #     for ax in axes:
    #         ax.set_rasterized(True)
    #     fig.tight_layout()
    fig.savefig(f"output/D={D}_seed={seed}_interactions.svg")
# fig, ax = show_matrix(agent_mat_pre, "Number of interactions", map="heatmap", sort=True, cmap="viridis", title="Pre RDN")
# fig, ax = show_matrix(agent_mat_post, "Number of interactions", map="heatmap", sort=True, cmap="viridis", title="Post RDN")


In [ ]:
vis = VisSocialNetwork(nsn)

kwargs = {"map": "heatmap", "sort": True}

# vis.show_adjacency_matrix(title="Pre RDN", t=(0.0, noise_start), **kwargs)
# vis.show_adjacency_matrix(title="Post RDN", t=(noise_start, noise_start+noise_length), **kwargs)
vis.show_adjacency_matrix(
    title="Pre RDN",
    t=(noise_start - 1.0, noise_start),
    cmap=sns.dark_palette(PRE_RDN_COLOR, as_cmap=True),
    **kwargs
)
vis.show_adjacency_matrix(
    title="Post RDN",
    t=(noise_start + noise_length - 1.0, noise_start + noise_length),
    cmap=sns.dark_palette(POST_RDN_COLOR, as_cmap=True),
    **kwargs
)


In [ ]:
columns = ["opinion", "in_degree"]
df_change = df_noise[columns] - df_start[columns]
df_change.columns = [
    math_fix(f"$\Delta {OPINION_SYMBOL}$"),
    math_fix(f"$\Delta {IN_DEGREE_SYMBOL}$"),
]
df_change[
    [
        math_fix(f"|$\Delta {OPINION_SYMBOL}$|"),
        math_fix(f"|$\Delta$ {IN_DEGREE_SYMBOL}|"),
    ]
] = df_change.abs()
df_change

In [ ]:
plot_scatter = True  # @param {type:"boolean"}
absolute_y = False  # @param {type:"boolean"}

with sns.plotting_context("paper"):
    plt.rcParams["figure.dpi"] = 200

    fig, axes = plt.subplots(
        ncols=2,
        figsize=(3.75, 2.25),
        gridspec_kw={"width_ratios": [1, 0.5]},
        sharey=False,
    )

    sns.kdeplot(
        data=df_change,
        x=df_change.columns[0],
        y=df_change.columns[1],
        color=DELTA_RDN_COLOR,
        fill=True,
        ax=axes[0],
        bw_adjust=0.5,
        thresh=0.05 * 2,
    )
    if plot_scatter:
        sns.scatterplot(
            data=df_change,
            x=df_change.columns[0],
            y=df_change.columns[1],
            color="grey",
            alpha=0.5,
            ec="None",
            s=1,
            ax=axes[0],
        )

    # absolutes

    sns.kdeplot(
        data=df_change,
        x=math_fix(f"|$\Delta {OPINION_SYMBOL}$|"),
        y=math_fix(f"|$\Delta$ {IN_DEGREE_SYMBOL}|"),
        color=DELTA_RDN_COLOR,
        ax=axes[1],
        fill=True,
        cut=0,
        bw_adjust=0.5,
        thresh=0.05 * 2,
    )
    if plot_scatter:
        #     sns.scatterplot(data=df_change, x=df_change.columns[2], y=df_change.columns[1+2*int(absolute_y)],
        #                     color="grey",
        #                     alpha=0.5, sn="None", s=1,
        #                     ax=axes[1]
        #                    )
        sns.regplot(
            data=df_change,
            x=math_fix(f"|$\Delta {OPINION_SYMBOL}$|"),
            y=math_fix(f"|$\Delta$ {IN_DEGREE_SYMBOL}|"),
            color="grey",
            scatter_kws=dict(
                alpha=0.5,
                ec="None",
                s=1,
            ),
            line_kws=dict(
                alpha=0.8,
                lw=2,
            ),
            ax=axes[1],
        )

    axes[0].set(
        xlim=(-4, 4), ylim=(-200, 200)
    )
    axes[1].set(xlim=(0, 2), ylim=(0, 200))
    plt.savefig("output/change_v_indegree.svg")

In [ ]:
# General params
N = 1000
m = 10
epsilon = 1e-2
gamma = 2.1
r = 0.5  # probability of mutual interaction
dt = 0.01
activity_distribution = negpowerlaw

# Specific params
param_set = {
    "polar": dict(K=3, alpha=3, beta=3),
}

T = 5.0
method = "RK45"
dt = 0.01
cache = "all"

D_range = [0.0, 3.0]
plt.rcParams["figure.dpi"] = 200
fig, axes = plt.subplot_mosaic(
    [
        [
            "time" if i == len(D_range) // 2 else ".",
            f"D panel {i}",
            f"distribution{i}",
            f"recovery{i}",
            f"distribution{i}recover",
        ]
        for i in range(len(D_range))
    ],
    figsize=(8, 8),
)

bins = 100
hist_kwargs = dict(
    title=False,
    vertical=False,
    alpha=0.5,
    bins=bins,
    kde=True,
    kde_kws=dict(bw_adjust=0.5),
    rasterized=True,
)
line_kwargs = dict(lw=0.05)
ylim = (-8, 8)

other_seeds = [1337, 91247, 712406]
time_points = [T, T * 2, T * 3]
x_df = pd.DataFrame(columns=["i", "D", "seed", OPINION_SYMBOL, "t"])
agent_idxs = list(range(N))

for i, D in enumerate(D_range):
    ax_opinion = axes[f"D panel {i}"]
    ax_dist = axes[f"distribution{i}"]
    ax_recovery = axes[f"recovery{i}"]
    ax_dist_recovery = axes[f"distribution{i}recover"]
    for s, seed in enumerate(other_seeds):
        with LoggingContext(logging.INFO):
            print(f"running D={D}")
            sn_polar = Simulation.run_params(
                SampleChamber,
                filename="sn_polar",
                N=N,
                m=m,
                **param_set["polar"],
                activity=activity_distribution,
                epsilon=epsilon,
                gamma=gamma,
                r=r,
                dt=dt,
                T=T,
                plot_opinion=False,
                method=method,
                D=0,
                cache=cache,
                seed=other_seeds[0], # start with the same seed each time
            )

            vis = VisSocialNetwork(sn_polar)

            if i == 0 and s == 0:
                vis.show_opinions(
                    ax=axes["time"],
                    color_code="line",
                    rasterized=True,
                    title=False,
                    **line_kwargs,
                )
                axes["time"].set(
                    xlim=(0, T),
                    ylim=ylim,
                )
            if s == 0:
                vis.show_opinions_distribution(
                    t=T, ax=ax_dist, color="Green", **hist_kwargs
                )

            sn_polar.filename = f"sn_polar_D{D:.2f}_[{seed}]"
            print(f"running {sn_polar.filename}")

            sn_polar.seed = seed

            if not (cache and sn_polar.load(dt, T * 2)):
                sn_polar.set_dynamics(D=D)
                sn_polar.run_network(dt, T, method=method)
                if cache:
                    sn_polar.save(*process_cache_arg(cache), write_mapping=False)

            vis.show_opinions(
                ax=ax_opinion,
                color_code="line",
                rasterized=True,
                title=False,
                **line_kwargs,
            )

            vis.show_opinions_distribution(ax=ax_dist, **hist_kwargs)

            sn_polar.filename = f"sn_polar_D{D:.2f}_recover[{seed}]"
            # turn off D
            if not (cache and sn_polar.load(dt, T * 3)):
                sn_polar.set_dynamics(D=0)
                sn_polar.run_network(dt, T, method=method)
                if cache:
                    sn_polar.save(*process_cache_arg(cache), write_mapping=False)

            vis.show_opinions(
                ax=ax_recovery,
                color_code="line",
                rasterized=True,
                title=False,
                **line_kwargs,
            )

            vis.show_opinions_distribution(ax=ax_dist_recovery, **hist_kwargs)

            for t in time_points:
                tmp_df = pd.DataFrame(
                    {
                        OPINION_SYMBOL: sn_polar.opinions_at_t(t)[-1],
                        "t": t,
                        "i": sn_polar.agent_idxs,
                        "D": D,
                        "seed": seed,
                    }
                )
                # concat
                x_df = pd.concat([x_df, tmp_df], ignore_index=True)

    sns.despine(ax=ax_dist, left=True)
    sns.despine(ax=ax_opinion, left=True)
    sns.despine(ax=ax_dist_recovery, left=True)
    sns.despine(ax=ax_recovery, left=True)
    # axes[1,i].axvline(x=T, color="k", linestyle="--")
    if i < len(D_range) - 1:
        ax_opinion.set(xlabel="", ylabel="", yticklabels=[], xticklabels=[])
        ax_recovery.set(xlabel="", ylabel="", yticklabels=[], xticklabels=[])
        ax_dist.set(xlabel="", ylabel="")
        ax_dist_recovery.set(xlabel="", ylabel="")
    ax_dist.set(xlim=ylim)
    ax_dist_recovery.set(xlim=ylim)
    ax_opinion.set(ylim=ylim)
    ax_opinion.set_xlim(left=T)
    ax_recovery.set(ylim=ylim)
    ax_recovery.set_xlim(left=T * 2)
    ax_opinion.set_title(f"D={D:.0f}", va="top", loc="left")
x_df = x_df.infer_objects()

In [ ]:
x_df = x_df.infer_objects()
g = sns.displot(
    data=x_df,
    x=OPINION_SYMBOL,
    hue="seed",
    palette="Purples",
    col="t",
    row="D",
    kind="kde",
    fill=True,
    bw_adjust=0.1,
)
g.set(xlim=(-5, 5))


### run a range of values

In [ ]:

seeds = np.geomspace(1, 100000, num=20, dtype=int)

noise_start = 5.0
noise_length = 5.0
recovery = 5.0
num = 1
interval = 0
time_points = [
    noise_start,
    noise_start + noise_length,
    noise_start + noise_length + recovery,
]

# Nudge arguments

D = 3  # @param {type:"slider", min:0, max:5, step:0.5}
sample_size = 30  # @param {type:"slider", min:0, max:50, step:1}
sample_method = "full"  # @param ["full", "subsample", "simple"]
seed = "1572038"  # @param ["1337", "10725983", "7988657", "1572039", "15379820"] {allow-input: true}
update_conn = True  # @param {type:"boolean"}
store_connections = False  # @param {type:"boolean"}

# polar opinions
N = 1000  # @param {type:"integer"}
m = 10  # @param {type:"integer"}
α = 3  # @param {type:"slider", min:0, max:5, step:0.5}
K = 3  # @param {type:"slider", min:0, max:5, step:0.5}
ε = 0.01  # @param {type:"slider", min:0.005, max:0.05, step:0.005}
Υ = 2.1  # @param {type:"slider", min:0, max:3, step:0.1}
β = 3  # @param {type:"slider", min:0, max:5, step:0.5}
r = 0.5  # @param {type:"slider", min:0, max:1, step:0.1}
dt = 0.01

integration_method = "RK45"  # @param ["Euler", "RK45"] {allow-input: true}

cls = "SampleChamber"  # @param ["SampleChamber", "OpenChamber"]

# could also use eval(cls)
cls_pick = {"SampleChamber": SampleChamber, "OpenChamber": OpenChamber}[cls]

kwargs = dict(
    N=N,
    m=m,
    activity_distribution=negpowerlaw,
    epsilon=ε,
    gamma=Υ,
    dt=dt,
    K=K,
    beta=β,
    alpha=α,
    r=r,
    update_conn=update_conn,
    store_all=store_connections,
    cls=cls_pick,
    method=integration_method,
    D=D,
    sample_size=sample_size,
    sample_method=sample_method,
    noise_start=noise_start,
    noise_length=noise_length,
    recovery=recovery,
    interval=interval,
    num=num,
)
range_params = {
    "seed": seeds[:10],
    # "K": [1, 2, 3],
    "D": [1, 3, 5],
    # "alpha": [1, 2, 3],
    "sample_method": ["full", "subsample"],
}

with LoggingContext(logging.WARNING):
    nsns = Simulation.run_product(
        range_params,
        **kwargs,
        name="delayed_noise",
        cache="all_opinion",
        cache_sim=False,
        cache_mem=True,
        parallel=10,
    )

    print("done running")
len(nsns)

In [ ]:
# create a dataframe of opinions at the special time points (Pre, Post, Recovery)
x_df = pd.DataFrame()
for nsn in nsns:
    D = max(_D for t, _D in nsn._D_hist)
    for t in time_points:
        tmp_df = pd.DataFrame(
            {
                OPINION_SYMBOL: nsn.opinions_at_t(t)[-1],
                "t": t,
                "D": D,
                "seed": nsn.seed,
                "sample_method": nsn._sample_method[0],
            }
        )
        # concat
        x_df = pd.concat([x_df, tmp_df], ignore_index=True)
print("done with x_df")


In [ ]:
x_df["seed"].unique()

In [ ]:
# calculate distribution_modality

data = x_df[(x_df["sample_method"].isin(["full"]))&(x_df["seed"].isin([]))]
dist = {
    (5.0, 0): [],
    (10.0, 1): [],
    (10.0, 3): [],
    (10.0, 5): [],
    (15.0, 0): [],
}
for key in dist.keys():
    t, D = key
    if D>0:
        dist[key] = distribution_modality(data[(data["t"]==t)&(data["D"]==D)][OPINION_SYMBOL].values)
    else:
         dist[key] = distribution_modality(data[(data["t"]==t)][OPINION_SYMBOL].values)

dist

In [ ]:
g = sns.displot(
    data=x_df[(x_df["sample_method"].isin(["full"]))&(~x_df["seed"].isin([]))],
    x=OPINION_SYMBOL,
    # hue="sample_method", hue_order=["full"],
    hue="t", palette=[PRE_RDN_COLOR, POST_RDN_COLOR, POST_RECOVERY_COLOR],
    # hue="seed",
    col="t",
    row="D",
    # kind="hist", stat="probability", kde=True, kde_kws=dict(bw_adjust=0.5,),
    kind="kde", fill=True, bw_adjust=0.5,
    height=3, 
)
g.set(xlim=(-5, 5), xticks=[-5, 0, 5], ylabel="", yticks=[])
g.set_titles("{row_name}")
sns.despine(left=True)
g.savefig(f"output/delayed_noise_panel.png", bbox_inches="tight")

In [ ]:
# plot a specific example
from opdynamics.visualise.vissimulation import show_periodic_noise

nsn = nsns[2]

# get D for this simulation (from the history)
# (goes from 0 to the actual value then back to 0)
D = max(_D for t, _D in nsn._D_hist)

show_periodic_noise(
    nsn,
    noise_start,
    noise_length,
    recovery,
    interval,
    num,
    D,
    t_window=0.0,
    lims=[-5, 5],
)

plt.savefig(f"output/delayed_noise_periodic.png", bbox_inches="tight")

## Range


In [ ]:
from opdynamics.utils.cache import get_cache_dir, set_cache_dir

old_dir, new_dir = set_cache_dir(os.path.join("/content", ".cache", "delayed_noise"))

noise_start = 10.0
noise_length = 10.0
recovery = 10.0
num = 1
interval = 0
seeds = [1337, 10725983, 7988657, 1572039, 15379820]
kwargs = dict(
    N=1000,
    m=10,
    activity_distribution=negpowerlaw,
    epsilon=1e-2,
    gamma=2.1,
    dt=0.01,
    K=3,
    beta=3,
    alpha=3,
    r=0.5,
    update_conn=True,
    cls=SampleChamber,
    method="RK45",
    D=5,
    sample_size=30,
    sample_method="full",
    noise_start=noise_start,
    noise_length=noise_length,
    recovery=recovery,
    interval=interval,
    num=num,
)

nec_list = Simulation.run_product(
    range_parameters={
        "D": {"range": [1, 3, 5]},
        "sample_size": {"range": [1, 30, 50]},
        "sample_method": {"range": ["full", "subsample", "simple"]},
        "seed": {"range": seeds},
    },
    cache="all",
    cache_sim=False,
    parallel=True,
    plot_opinion=True,
    **kwargs
)

old_dir, new_dir = set_cache_dir(old_dir)

nec_dict = {}
for nsn in nec_list:
    key = (tuple(nsn._D_hist), nsn._sample_method[0], nsn._sample_size)
    nec_dict.setdefault(key, [])
    nec_dict[key].append(nsn)

In [ ]:
from tqdm.contrib import tenumerate
from matplotlib.cbook import flatten

num_seeds = len(seeds)

with sns.plotting_context("paper"):
    plt.rcParams["axes.titlesize"] = "xx-small"
    plot_colors = sns.color_palette("Set1", n_colors=num_seeds)
    fig, axs = plt.subplots(
        len(nec_dict.keys()),
        num_seeds + 1,
        figsize=(5, 15),
        sharex="col",
        sharey="all",
    )
    fig.subplots_adjust(hspace=0.3)
    sns.despine(fig, bottom=True)
    for i, key in tenumerate(sorted(nec_dict.keys())):
        print(key)
        for j, nsn in tenumerate(nec_dict[key]):
            _visec = VisSocialNetwork(nsn)
            title_key = f"{key[0][1][1]}, {key[1]}, {key[2]}"
            _visec.show_opinions(
                "line", subsample=50, title=title_key, fig=fig, ax=axs[i, j]
            )
            _visec.show_opinions_distribution(
                t=20,
                vertical=True,
                color=plot_colors[j],
                ax=axs[i, -1],
                title="",
                element="poly",
                stat="density",
            )
            axs[i, j].axvline(20, ymin=-10, ymax=10, lw=0.5, c=plot_colors[j])
            axs[i, j].axvline(10, ymin=-10, ymax=10, ls="--", lw=0.5, c=plot_colors[j])
    for ax in flatten(axs[:, 1:]):
        ax.set_ylabel("")
    for ax in flatten(axs[:-1, :]):
        ax.set_xlabel("")

    axs[0, 0].set_ylim(-5, 5)
print("showing")
